63f78014766fd30436c18a79_Hyperspace - navbar logo.png

# Movie Recommendation System Using Hyperspace Engine
This notebook demonstrates the use of Hyperspace engine to create a movie recommendation system, by combining classic search (keyword matching) with vector search. The notebook first demonstrates classic and vector search, followed by hybrid search, which combines classic and vector searches, by combining word embedding with metadata filtering.

Finally, we will demonstrate the use of aggregation to analyze data regarding the returned documents.

# The Dataset
The data is taken from [MovieLens Latest Datasets](https://grouplens.org/datasets/movielens/latest/) and was downloaded from [Kaggle movie recommender system dataset ](https://www.kaggle.com/code/rounakbanik/movie-recommender-systems). The data includes 40951 valid movies. The data is in SQL format (table) and will be converted to NoSQL (documents) format. The data preprocessing is given in the notebook titles "MovieRecommendationDataPrep", available in this repository.

## The Dataset Fields
The processed metadata includes the following fields:

1.   **adult** [boolean] - states if the movie is rated 18+
2.   **belongs_to_collection** [Keyword] - name of the collection that includes the movie. If the movie is not a part of a collection, value will be "None"
3. **budget** [integer] - The budget of the movie in USD
4. **genres** [list[Keyword]] - list of movie genres (i.e drama)
5. **id** [integer]] - unique id per movie
6. **original_language** [Keyword] - the original language in which the movie was produced
7. **popularity** [float] - the popularity of the movie, formulated as an unbounded score
8. **production_companies** [list[Keyword]] - list of production companies involved in the movie
9. **production_countries** [list[Keyword]] - list of all countries in which the movie was filmed
10. **rating** [float] - the movie IMDB weighted average rating  score
11. **release_date_unix_time** [int] - the movie release date in unix time
12. **revenue** [float] - the movie rvenue in [USD]
13. **runtime_days** [int] - the number cinema run time days
14. **spoken_languages** [list[Keyword]] - list of all languages spoken in the movie
15. **title** [Keyword] - the movie title
16. **description embedding** [list[float]] - embedding of the movie tagline and description text


# Loading the Data
We first load the movie metadata in an SQL from from a csv file, using the pandas module. The data was previously processed in order to only include the relevant features. The data does not include vectors, and we will create them usin simple TF-IDF embedding, in the next step.


In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')
df = pd.read_csv('Movie_Recommendation_Processed.csv')
df["runtime_days"] = df["runtime_days"].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40951 entries, 0 to 40950
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  40951 non-null  int64  
 1   adult                  40951 non-null  bool   
 2   belongs_to_collection  40951 non-null  object 
 3   budget                 40951 non-null  int64  
 4   genres                 40951 non-null  object 
 5   id                     40951 non-null  int64  
 6   overview               40951 non-null  object 
 7   popularity             40951 non-null  float64
 8   production_companies   40951 non-null  object 
 9   production_countries   40951 non-null  object 
 10  revenue                40951 non-null  float64
 11  runtime_days           40951 non-null  int64  
 12  spoken_languages       40951 non-null  object 
 13  tagline                18454 non-null  object 
 14  title                  40951 non-null  object 
 15  ye

# Word Embedding
The next step is to embedd the text of movie overviews and the taglines. We use a simple TF-IDF based vectorization  (in contrast to a more sophisticated embedding using,i.e., BERT or GPT) using the SKLEARN TfidfVectorizer. The first step will be to normalize the text and then replace rare words with base tense

In [ ]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from pprint import pprint, pformat

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stopwords = list(stopwords.words('english'))
replacement_dict = {"Cheated":"Cheat","Photographs":"Photograph","Awfully":"Awful","Poisoner":"Poisoner","comix":"comics",
                    "embarrassingly":"embarrassing"}

def normalize_text(tagline):
    tagline = re.sub(r'\W', ' ', tagline)

    words = nltk.word_tokenize(tagline)
    normalized_words = [word for word in words if word.lower() not in ['be', 'is', 'are', 'am', 'was', 'were', 'been', 'being'] + stopwords]
    normalized_words = [lemmatizer.lemmatize(word, pos='v') for word in normalized_words if len(word) > 1 and not word.isdigit()]

    normalized_tagline = ' '.join(normalized_words)
    for key in replacement_dict:
      normalized_tagline = normalized_tagline.replace(key,replacement_dict[key])
    return normalized_tagline

df['tagline'] = df['tagline'].fillna("''")
df['tagline'] = df['tagline'].apply(normalize_text)
df['overview'] = df['overview'].apply(normalize_text)
replacement_dict = {"Cheated":"Cheat","Photographs":"Photograph","Awfully":"Awful","Poisoner":"Poisoner","comix":"comics",
                    "embarrassingly":"embarrassing"}


df["description text"] = df["overview"] + df["tagline"]
del(df["overview"])
del(df["tagline"])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
import math
embedding_vector_length = 2048

def normalize_euclidean_norm(input_list):
    euclidean_norm = math.sqrt(sum(x ** 2 for x in input_list))
    normalized_list = [x / euclidean_norm for x in input_list]
    return normalized_list

def embedded_text(df_data, min_word_count):
  tfidf = TfidfVectorizer(token_pattern=r'\b\w+\b', stop_words="english", min_df=min_word_count)
  tfidf_matrix = tfidf.fit_transform(df_data)
  idf_values = tfidf.idf_
  top_terms_idx = idf_values.argsort()[::-1][:embedding_vector_length]
  top_terms = [list(tfidf.vocabulary_.keys())[i] for i in top_terms_idx]
  new_tfidf = TfidfVectorizer(vocabulary=top_terms)
  new_tfidf_matrix = new_tfidf.fit_transform(df_data)
  new_tfidf_matrix = round(100 * new_tfidf_matrix)/100
  tfidf_matrix = new_tfidf_matrix.toarray()
  new_col = df_data.copy()
  return list(tfidf_matrix), new_tfidf, top_terms

df["description embedding"], tfidf_model, top_terms = embedded_text(df["description text"], 10)
df["description embedding"] = df["description embedding"].map(lambda x: list(x))
df["description embedding"] = df["description embedding"].map(lambda x: normalize_euclidean_norm(x))
df.reset_index(inplace=True)


In [ ]:
import math
embedding_vector_length = 2048

def normalize_euclidean_norm(input_list):
    euclidean_norm = math.sqrt(sum(x ** 2 for x in input_list))
    normalized_list = [x / euclidean_norm for x in input_list]
    return normalized_list

df["description embedding"] = df["description embedding"].map(lambda x: normalize_euclidean_norm(x))


In order to allow the user to include additional text, we include the function "embed_text" which prfoms the embedding process for given text

In [ ]:
def embed_text(text):
  text = normalize_text(text)
  text = text.split(" ")
  vec = [0] * embedding_vector_length
  for word in text:
    for i, term in enumerate(top_terms):
      if word == term:
        vec[i] = vec[i]+1
        continue
  return normalize_euclidean_norm(vec)


1.0

In [ ]:
import numpy as np
np.sum(np.array(df["description embedding"][45]))

2.18

# Using the Hyperspace Engine
In the next step will Setting the environment requires the following steps


1. Install the client API
2. Connect to a server
3. Create data schema file
4. Create collection
5. Ingest data
6. Run query

## 1. Install the client API
Hyperspace API can be installed directly from git, using the following command:

In [ ]:
pip install git+https://github.com/hyper-space-io/hyperspace-py

## 2. Connect to a server

Once the Hyperspace API is installed, the database can be accessed by creating a local instance of the Hyperspace client. This step requires host address, username and password.

In [ ]:
import hyperspace

hyperspace_client = hyperspace.HyperspaceClientApi(host='https://search-master-demo.development.hyper-space.xyz',
                                                   username=username, password=password)


Before continuing, let us check that the cluster is live using the  "**cluster_status()**" command

In [ ]:
hyperspace_client.cluster_status()

## 3.   Create a Data Schema File

Similarly to other search databases, Hyper-Space database requires a configuration file that outlines the data schema. Here, we create a config file that corresponds to the fields of the given dataset.

For vector fields, we also provide the index type to be used, and the metric. . Current options for index include "**brute_force**", "**hnsw**", "**ivf**", and "**bin_ivf**" for binary vectors, and "**IP**" (inner product) as a metric for floating point vectors and "**Hamming**" ([hamming distance](https://en.wikipedia.org/wiki/Hamming_distance)) for binary vectors.

Here, we use standrad inner product (metric = IP) with exact KNN (index = brute_force).

In [ ]:
import json

config = {
  "configuration": {
    "adult": {
      "type": "boolean"
    },
    "belongs_to_collection": {
      "type": "keyword"
    },
    "budget": {
      "type": "integer"
    },
    "genres": {
      "struct_type": "list",
      "type": "keyword"
    },
    "id": {
      "type": "integer"
    },
    "original_language": {
      "type": "keyword"
    },
    "popularity": {
      "type": "float"
    },
    "production_companies": {
      "struct_type": "list",
      "type": "keyword"
    },
    "production_countries": {
      "struct_type": "list",
      "type": "keyword"
    },
    "rating": {
      "type": "float"
    },
    "release_date_unix_time": {
      "type": "date"
    },
    "revenue": {
      "type": "float"
    },
    "runtime_days": {
      "type": "integer"
    },
    "spoken_languages": {
      "struct_type": "list",
      "type": "keyword"
    },
    "title": {
      "type": "keyword"
    },
     "description embedding": {
            "type": "dense_vector",
            "dim": 2048,
            "index_type": "brute_force",
            "metric": "IP"
      }
  }
}

with open('MovieRecommendation_config.json', 'w') as f:
    f.write(json.dumps(config, indent=2))



## 4. Create Collection
The Hyerspace engine stroes data in Collections, where each collecction commonly hosts data of similar context, etc. Each search is then perfomed within a collection. We create a collection using the command "**create_collection**(schema_filename, collection_name)".

In [ ]:
delete_collections = True
collection_name = 'Movies'

if delete_collections:
  try:
    hyperspace_client.delete_collection(collection_name)
  except:
    pass

hyperspace_client.create_collection('MovieRecommendation_config.json', collection_name)
hyperspace_client.cluster_status()

## 5. Ingest data

In the next step we ingest the dataset in batches of 250 documents. This number can be controlled by user, and in particular, can be increased in order improve ingestion time. We add batches of data using the command **add_batch**(batch, collection_name).

In [ ]:
import time
BATCH_SIZE = 500

def chunker(df, size):
    return (df.iloc[pos:pos + size] for pos in range(0, len(df), size))


start = time.time()

i = 0

for chunk in chunker(df.iloc[i:], BATCH_SIZE):
    batch = [hyperspace.Document(str(i + j), row) for j, row in enumerate(chunk.to_dict('records'))]
    i += BATCH_SIZE

    if i % BATCH_SIZE == 0:
        response = hyperspace_client.add_batch(batch, collection_name)
        print(i, response)
        batch.clear()


hyperspace_client.commit('Movies')
time = (time.time() - start)
print(f"ingestion took: {time} sec")

500 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
1000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
1500 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
2000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
2500 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
3000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
3500 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
4000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
4500 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
5000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
5500 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
6000 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
6500 {'code': 200, 'message': 'Batch successfully added', 'status': 'OK'}
7000 {'code': 200, 'message': 'Batch su

## 6. Running The Query

### Creating The Query
Hyperspace queries are created in python format and saved as strings. Classic search requires a pre-created score function.

### Loading the score function
The score function encorporates logic based on movied budget and rating, and gives bonus to movies of similar production_companies. Only movies of the same genre are returned. We will use two score functions. The first is used only for matching similar movies, while the other will perform agrregrations on the returned data.

In [ ]:
sf_file = 'movies_score_function.py'
hyperspace_client.set_function(sf_file, collection_name=collection_name, function_name='popular_films_recommendation')

In [ ]:
sf_agg_file = 'movie_aggregation_function.py'
hyperspace_client.set_function(sf_agg_file, collection_name=collection_name, function_name='aggregation_films_recommendation')

The next step is use the query logic and apply the query. We will run a vector search, followed by a hybrid search which includes analytic logic - boost based on rating, genres, etc. Let's start with vector search.

### Vector Search
We start with a simple search function, based on matching of embedded vectors using inner product and accurate KNN

In [ ]:
input_vector = hyperspace_client.get_document(document_id='555', collection_name=collection_name)
pprint({x: input_vector[x] for x in input_vector if x!= "description embedding"})

{'adult': False,
 'belongs_to_collection': 'None',
 'budget': 38000000,
 'description text': 'young orphan boy James spill magic bag crocodile tongue '
                     'find possession giant peach fly away strange '
                     'landAdventures big grow tree',
 'genres': "['Adventure', 'Animation', 'Family']",
 'id': 10539,
 'index': 651,
 'level_0': 555,
 'popularity': 13.217623,
 'production_companies': "['Walt Disney Pictures', 'Allied Filmmakers']",
 'production_countries': "['US']",
 'rating': 5.692732773656594,
 'revenue': 28921264.0,
 'runtime_days': 79,
 'spoken_languages': "['en']",
 'title': 'James and the Giant Peach',
 'unix_time': 829267200,
 'year': 1996}


In [ ]:
query = {
    'params': input_vector,
    "knn": {
        "query": {"boost": 0}, # boost = 0 , means no metadata filtering
        "description embedding": {
            "boost": 1,
        }
    }
}

print("searching for matches for '",input_vector["title"],"'")
print("-------------------------------------------------")
results = hyperspace_client.search(query,
                                        size=10,
                                        function_name='popular_films_recommendation',
                                        collection_name=collection_name)

candidates = results['candidates']

print(f"Query run time = {results['took_ms']}ms")
print("-------------------------------------------------")


print("{:<5} {:<10} {:<40} {:<10}".format("Rank", "ID", "Title", "Score"))
print("="*65)

for i, result in enumerate(results['similarity']):
    api_response = hyperspace_client.get_document(document_id=result['document_id'], collection_name=collection_name)
    print("{:<5} {:<10} {:<40} {:<10}".format(i + 1, result['document_id'], api_response['title'][:40], round(result['score'], 2)))


searching for matches for ' James and the Giant Peach '
-------------------------------------------------
Query run time = 99.1371ms
-------------------------------------------------
Rank  ID         Title                                    Score     
1     555        James and the Giant Peach                1.01      
2     36676      Air Mater                                0.71      
3     20863      A Kid for Two Farthings                  0.7       
4     40550      The Famous Box Trick                     0.7       
5     15758      Mystics in Bali                          0.67      
6     23548      Magic Boy                                0.66      
7     9331       Freedomland                              0.61      
8     454        Sleepless in Seattle                     0.61      
9     19950      Gamera vs. Viras                         0.6       
10    16095      Magic Christmas Tree                     0.58      


### Classic Search
Let us use thescore function, and run classic search (keyword matching).

In [ ]:
input_vector = hyperspace_client.get_document(document_id='555', collection_name=collection_name)

print("searching for matches for '",input_vector["title"],"'")
print("-------------------------------------------------")

query = {
    'params': input_vector,
    "knn": {
        "query": {"boost": 2}, # boost = 0 , means no metadata filtering
    }
}

results = hyperspace_client.search(query,
                                        size=10,
                                        function_name='popular_films_recommendation',
                                        collection_name=collection_name)

candidates = results['candidates']

print(f"Query run time = {results['took_ms']}ms")
print("------------------------------------------------------")


print("{:<5} {:<10} {:<40} {:<10}".format("Rank", "ID", "Title", "Score"))
print("="*65)

for i, result in enumerate(results['similarity']):
    api_response = hyperspace_client.get_document(document_id=result['document_id'], collection_name=collection_name)
    print("{:<5} {:<10} {:<40} {:<10}".format(i + 1, result['document_id'], api_response['title'][:40], round(result['score'], 2)))



searching for matches for ' James and the Giant Peach '
-------------------------------------------------
Query run time = 3.81724ms
------------------------------------------------------
Rank  ID         Title                                    Score     
1     37206      Moana                                    360.14    
2     27368      The Good Dinosaur                        280.32    
3     11584      The Tale of Despereaux                   280.24    
4     14703      Mars Needs Moms                          280.14    
5     36753      Trolls                                   280.14    
6     30255      Phantom Boy                              240.6     
7     36977      Little Longnose                          240.37    
8     16752      Jack-Jack Attack                         240.14    
9     21511      No Time for Nuts                         240.14    
10    22010      The Little Fox                           240.14    


### Hybrid Search
In the next step we apply Hybrid search, which combines classic and vector search.

In [ ]:
input_vector = hyperspace_client.get_document(document_id='555', collection_name=collection_name)

print("searching for matches for '",input_vector["title"],"'")
print("-------------------------------------------------")

query = {
    'params': input_vector,
    "knn": {
        "query": {"boost": 1},
        "description embedding": {
            "boost": 200,
        }
    }
}

results = hyperspace_client.search(query,
                                        size=10,
                                        function_name='aggregation_films_recommendation',
                                        collection_name=collection_name)

candidates = results['candidates']

print(f"Query run time = {results['took_ms']}ms")
print("------------------------------------------------------")


print("{:<5} {:<10} {:<40} {:<10}".format("Rank", "ID", "Title", "Score"))
print("="*65)

for i, result in enumerate(results['similarity']):
    api_response = hyperspace_client.get_document(document_id=result['document_id'], collection_name=collection_name)
    print("{:<5} {:<10} {:<40} {:<10}".format(i + 1, result['document_id'], api_response['title'][:40], round(result['score'], 2)))

searching for matches for ' James and the Giant Peach '
-------------------------------------------------
Query run time = 4.24484ms
------------------------------------------------------
Rank  ID         Title                                    Score     
1     30255      Phantom Boy                              210.87    
2     37206      Moana                                    180.07    
3     27368      The Good Dinosaur                        176.07    
4     36977      Little Longnose                          164.59    
5     11584      The Tale of Despereaux                   159.87    
6     14703      Mars Needs Moms                          140.07    
7     36753      Trolls                                   140.07    
8     16752      Jack-Jack Attack                         120.07    
9     21511      No Time for Nuts                         120.07    
10    22010      The Little Fox                           120.07    


# Hybrid Search and Agggregations
In the final part, we will use a different score function to aggregatre various fields of the data. The aggregation is perfomed in the score function, where each aggregation is perfomed over all candidates that passed the filtering up to the step that hosts it.

In [ ]:
input_vector = hyperspace_client.get_document(document_id='555', collection_name=collection_name)

print("searching for matches for '",input_vector["title"],"'")
print("-------------------------------------------------")

query = {
    'params': input_vector,
    "knn": {
        "query": {"boost": 1},
        "description embedding": {
            "boost": 200,
        }
    }
}

results = hyperspace_client.search(query,
                                        size=10,
                                        function_name='aggregation_films_recommendation',
                                        collection_name=collection_name)

candidates = results['candidates']

print(f"Query run time = {results['took_ms']}ms")
print("------------------------------------------------------")


print("{:<5} {:<10} {:<40} {:<10}".format("Rank", "ID", "Title", "Score"))
print("="*65)

for i, result in enumerate(results['similarity']):
    api_response = hyperspace_client.get_document(document_id=result['document_id'], collection_name=collection_name)
    print("{:<5} {:<10} {:<40} {:<10}".format(i + 1, result['document_id'], api_response['title'][:40], round(result['score'], 2)))


print("\n----------------- Aggregations --------------------")
print("{:<50} {:<15}".format("Aggregation", "Value"))
print("="*55)

for i, agg in enumerate(results['aggregations']):
    if results['aggregations'][agg]['agg_type'] == "count":
        print("{:<50} {:<15}".format(agg, results['aggregations'][agg]["count"]))
    else:
        print("{:<50} {:<15}".format(agg, round(results['aggregations'][agg]["value"],2)))

searching for matches for ' James and the Giant Peach '
-------------------------------------------------
Query run time = 5.30331ms
------------------------------------------------------
Rank  ID         Title                                    Score     
1     30255      Phantom Boy                              210.87    
2     37206      Moana                                    180.07    
3     27368      The Good Dinosaur                        176.07    
4     36977      Little Longnose                          164.59    
5     11584      The Tale of Despereaux                   159.87    
6     14703      Mars Needs Moms                          140.07    
7     36753      Trolls                                   140.07    
8     16752      Jack-Jack Attack                         120.07    
9     21511      No Time for Nuts                         120.07    
10    22010      The Little Fox                           120.07    

----------------- Aggregations --------------------


63f78014766fd30436c18a79_Hyperspace - navbar logo.png